# Emplifi API Exploration - Debugging Syngenta Flowers Query

This notebook helps debug why the Syngenta Flowers query (ID: LNQ_1140092_641afbbd98a766f5eb4a4915) is returning 0 posts and 0 metrics data points. We'll systematically test the API to understand what data is available and why our integration test shows no results.

## Import Required Libraries

Import necessary libraries including our custom Emplifi API modules, pandas for data analysis, and datetime for date range testing.

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import json
from datetime import datetime, timedelta, timezone
from typing import Any, Dict, List

# Add the src directory to the path so we can import our modules
notebook_dir = Path().resolve()
src_dir = notebook_dir.parent / "src"
sys.path.insert(0, str(src_dir))

# Import our Emplifi tools
from mcp_server.tools.emplifi_tools import (
    list_listening_queries,
    fetch_listening_posts,
    fetch_listening_metrics,
    get_recent_posts,
    get_daily_mention_metrics,
    MetricConfig,
    DimensionConfig,
    PostsSort,
    PostsFilter,
    _get_auth_client,
    _get_config
)

print("✅ Libraries imported successfully")
print(f"📁 Working from: {notebook_dir}")
print(f"📦 Source directory: {src_dir}")

✅ Libraries imported successfully
📁 Working from: /home/gferreir/mines/mcp/mcp_template/notebooks
📦 Source directory: /home/gferreir/mines/mcp/mcp_template/src


## Setup API Configuration

Configure API credentials and test the authentication to ensure we can connect to the Emplifi API.

In [2]:
# Test API configuration
try:
    config = _get_config()
    print("📋 Configuration loaded:")
    print(f"   OAuth Client ID: {'✅ Set' if config.client_id else '❌ Not set'}")
    print(f"   Basic Token: {'✅ Set' if config.basic_token else '❌ Not set'}")
    print(f"   Basic Secret: {'✅ Set' if config.basic_secret else '❌ Not set'}")
    
    # Test authentication
    auth_client = _get_auth_client()
    headers = await auth_client.get_auth_headers()
    
    auth_type = "Basic" if config.basic_token else "OAuth"
    print(f"\n🔐 Authentication: {auth_type}")
    print(f"   Headers obtained: {'✅ Success' if headers else '❌ Failed'}")
    
except Exception as e:
    print(f"❌ Configuration error: {e}")
    print("\n💡 Make sure your .env file contains either:")
    print("   - EMPLIFI_TOKEN and EMPLIFI_SECRET (for Basic auth)")
    print("   - Or EMPLIFI_CLIENT_ID, EMPLIFI_CLIENT_SECRET, EMPLIFI_REDIRECT_URI (for OAuth)")

📋 Configuration loaded:
   OAuth Client ID: ❌ Not set
   Basic Token: ✅ Set
   Basic Secret: ✅ Set

🔐 Authentication: Basic
   Headers obtained: ✅ Success


## Retrieve All Listening Queries

Fetch all available listening queries to verify the total count and examine the query structure.

In [3]:
# Fetch all listening queries
try:
    print("🔍 Fetching all listening queries...")
    queries = await list_listening_queries()
    
    print(f"\n📊 Total queries found: {len(queries)}")
    
    # Convert to DataFrame for better analysis
    if queries:
        query_data = []
        for q in queries:
            query_data.append({
                'id': q.id,
                'name': q.name,
                'description': q.description,
                'status': q.status
            })
        
        df_queries = pd.DataFrame(query_data)
        
        print("\n📋 Query Status Summary:")
        status_counts = df_queries['status'].value_counts()
        for status, count in status_counts.items():
            print(f"   {status}: {count} queries")
        
        print("\n📝 Sample queries (first 5):")
        display(df_queries.head())
        
        # Store for later use
        all_queries = queries
        
    else:
        print("❌ No queries found")
        
except Exception as e:
    print(f"❌ Error fetching queries: {e}")
    import traceback
    print("\n🔍 Full error details:")
    traceback.print_exc()

🔍 Fetching all listening queries...

📊 Total queries found: 286

📋 Query Status Summary:
   Running: 257 queries
   Regulated: 13 queries
   Paused: 11 queries
   Done: 5 queries

📝 Sample queries (first 5):

📊 Total queries found: 286

📋 Query Status Summary:
   Running: 257 queries
   Regulated: 13 queries
   Paused: 11 queries
   Done: 5 queries

📝 Sample queries (first 5):


,id,name,description,status
0,LNQ_1140092_641afbbd98a766f5eb4a4915,Syngenta Flowers,None,Running
1,LNQ_1140092_6433af2612abdcc89485af76,Canada Seedcare Products,None,Running
2,LNQ_1140092_644a178fa2cd1efe2f4427b6,Bettina PQT 2025,None,Running
3,LNQ_1140092_6450290f66aaefdf461ee521,Canada Herbicides,None,Running
4,LNQ_1140092_64502c2d85a9bb0da5823d90,Canada Fungicides,None,Running


In [4]:
# save df_queries to path /home/gferreir/mines/emplifi_listening_api/data
# df_queries.to_csv("/home/gferreir/mines/emplifi_listening_api/data/emplifi_listening_queries.csv", index=False)



~~ ## Find Syngenta Flowers Query ~~ 

Search for and examine the specific Syngenta Flowers query to understand its configuration.

In [5]:
# Constants for our target query
# 94	('id', '')	('name', 'BAYER ')	('description', None)	('status', 'Running')
TARGET_QUERY_ID = "LNQ_1140092_66fe2dcd3e9eb298096e8db3"
TARGET_QUERY_NAME = "BAYER "

# TARGET_QUERY_ID = "LNQ_1140092_641afbbd98a766f5eb4a4915"
# TARGET_QUERY_NAME = "Syngenta Flowers"

# Find the Syngenta Flowers query
syngenta_query = None
if 'all_queries' in locals():
    for query in all_queries:
        if query.id == TARGET_QUERY_ID:
            syngenta_query = query
            break

if syngenta_query:
    print(f"✅ Found {TARGET_QUERY_NAME} query!")
    print(f"   ID: {syngenta_query.id}")
    print(f"   Name: {syngenta_query.name}")
    print(f"   Description: {syngenta_query.description or 'None'}")
    print(f"   Status: {syngenta_query.status}")
    
    # Check if name matches expected
    if syngenta_query.name == TARGET_QUERY_NAME:
        print("✅ Name matches expected value")
    else:
        print(f"⚠️  Name mismatch: expected '{TARGET_QUERY_NAME}', got '{syngenta_query.name}'")
    
    # Check status
    if syngenta_query.status and syngenta_query.status.lower() in ['running', 'active']:
        print("✅ Query appears to be active")
    else:
        print(f"⚠️  Query status may be inactive: {syngenta_query.status}")
        
else:
    print(f"❌ {TARGET_QUERY_NAME} query not found!")
    print(f"   Looking for ID: {TARGET_QUERY_ID}")
    print(f"   Looking for name: {TARGET_QUERY_NAME}")
    
    # Search by name in case ID changed
    print("\n🔍 Searching by name...")
    name_matches = [q for q in all_queries if 'syngenta' in q.name.lower() or 'flower' in q.name.lower()]
    if name_matches:
        print(f"   Found {len(name_matches)} potential matches:")
        for q in name_matches:
            print(f"     - {q.name} ({q.id}) - Status: {q.status}")
    else:
        print("   No queries found containing 'syngenta' or 'flower'")

✅ Found BAYER  query!
   ID: LNQ_1140092_66fe2dcd3e9eb298096e8db3
   Name: BAYER 
   Description: None
   Status: Running
✅ Name matches expected value
✅ Query appears to be active


_____________________________

In [6]:
# read the df
df_queries = pd.read_csv("/home/gferreir/mines/emplifi_listening_api/data/emplifi_listening_queries.csv")

TARGET_QUERY_ID = "LNQ_1140092_66fe2dcd3e9eb298096e8db3"
TARGET_QUERY_NAME = "BAYER "

## Test Posts Retrieval

Attempt to retrieve posts

In [7]:
start_date = "2025-01-01"
end_date = "2025-12-31"

posts = await fetch_listening_posts(
                query_ids=[TARGET_QUERY_ID],
                date_start=start_date,
                date_end=end_date,
                limit=10,  # Small limit for testing
                max_pages=1
            )

posts_results = len(posts)
print(f"   📊 Found {len(posts)} posts")

2025-08-12 19:59:56 [info     ] Fetching posts with payload: {'listening_queries': ['LNQ_1140092_66fe2dcd3e9eb298096e8db3'], 'date_start': '2025-01-01', 'date_end': '2025-12-31', 'limit': 10, 'fields': ['id', 'created_time', 'platform', 'author', 'message', 'sentiment', 'interactions', 'url'], 'sort': [{'field': 'interactions', 'order': 'desc'}]}
2025-08-12 19:59:57 [info     ] API response structure: ['success', 'data']
2025-08-12 19:59:57 [info     ] Page 1: Found 10 posts        
2025-08-12 19:59:57 [info     ] Successfully fetched 10 posts 
   📊 Found 10 posts
2025-08-12 19:59:57 [info     ] API response structure: ['success', 'data']
2025-08-12 19:59:57 [info     ] Page 1: Found 10 posts        
2025-08-12 19:59:57 [info     ] Successfully fetched 10 posts 
   📊 Found 10 posts


In [17]:
# convert posts to pandas dataframe:
# ListeningPost(id='18054880931182875', created_time='2025-04-12T11:00:00+00:00', platform='instagram', author={'id': '17841400993243858', 'name': 'Mark Hyman, M.D.', 'url': 'https://www.instagram.com/drmarkhyman'}, message='Not all produce is created equal.\n\nThanks to companies like Bayer (who bought Monsanto, the makers of Roundup), our food system is flooded with pesticides—many of which are linked to hormone disruption, neurological damage, and cancer. \n\nIn fact, the main chemical in Roundup—glyphosate—was classified as a probable human carcinogen by the World Health Organization. And yet, it’s still sprayed on crops all over the U.S.\n\nHere’s the good news:\nYou can reduce your exposure dramatically by following the EWG’s Dirty Dozen and Clean 15 lists.\n\n🧪 Dirty Dozen = the fruits and veggies with the highest pesticide residues. Always try to buy these organic if you can afford it. Note: these are still better than any ultra processed food.\n🥑 Clean 15 = produce with natural protection (like a peel or husk), and generally safe to buy conventional.\n\nGeneral rule: If it has a thick peel—like an avocado or banana—it’s likely safer. If you eat the whole thing—like spinach or strawberries—go organic.\n\nSave this for your next grocery trip. Small swaps = big impact on your health.', sentiment='negative', interactions=50864, url=None)

df_posts = pd.DataFrame([{
    "id": post.id,
    "created_time": post.created_time,
    "platform": post.platform,
    "author_id": post.author.get("id"),
    "author_name": post.author.get("name"),
    "author_url": post.author.get("url"),
    "message": post.message,
    "sentiment": post.sentiment,
    "interactions": post.interactions,
    "url": post.url
} for post in posts])

df_posts

,id,created_time,platform,author_id,author_name,author_url,message,sentiment,interactions,url
0,18054880931182875,2025-04-12T11:00:00+00:00,instagram,17841400993243858,"Mark Hyman, M.D.",https://www.instagram.com/drmarkhyman,Not all produce is created equal.\n\nThanks to...,negative,50864,None
1,18062456180184551,2025-07-11T10:52:30+00:00,instagram,17841404666905163,VERSUS,https://www.instagram.com/versus,New Balance and Bayer Leverkusen have gone two...,neutral,39783,None
2,18060552065069111,2025-03-19T17:26:48+00:00,instagram,17841400993243858,"Mark Hyman, M.D.",https://www.instagram.com/drmarkhyman,Drop a YES👇for a link to this full conversatio...,negative,29395,None
3,UmAJrhsLub8,2025-08-03T10:00:43+00:00,youtube,UC3w193M5tYPJqF0Hi-7U-2g,Dr. Eric Berg DC,https://www.youtube.com/channel/UC3w193M5tYPJq...,Fight back against the pesticide immunity bill...,negative,25959,None
4,1922969659550396664,2025-05-15T10:57:48+00:00,twitter,None,None,None,INSANITY 🚨 Georgia Governor Brian Kemp SIGNED ...,negative,21914,None
5,1948048374286180852,2025-07-23T15:51:40+00:00,twitter,None,None,None,"Well, Section 453 was approved yesterday. \n\n...",neutral,19716,None
6,18271974751271940,2025-02-18T17:45:45+00:00,instagram,17841401569094025,Vani Hari | Food Babe,https://www.instagram.com/thefoodbabe,Bayer wants to keep this out of the public eye...,negative,18361,None
7,18042304307098097,2025-02-15T11:00:00+00:00,instagram,17841400993243858,"Mark Hyman, M.D.",https://www.instagram.com/drmarkhyman,"Glyphosate, the active ingredient in Roundup, ...",neutral,15345,None
8,108525343979507_1023818919836379,2025-06-15T13:33:49+00:00,facebook,108525343979507,Bayer Crop Science Pakistan,https://www.facebook.com/BayerCropSciencePakistan,دوست وہی جو کام آئے \nبائر کا اوبیرون اسپیڈ\n\...,neutral,15322,None
9,VZm4C1tiQlM,2025-04-05T10:28:47+00:00,youtube,UC1DtEMePmr4O6F2do6BVl7A,Rahul Gandhi,https://www.youtube.com/channel/UC1DtEMePmr4O6...,I recently met a family that runs HP Singh Fab...,neutral,13172,None


## Test Metrics Retrieval

In [19]:
metric_type = "authors"

metrics = await fetch_listening_metrics(
                    query_ids=[TARGET_QUERY_ID],
                    date_start=start_date,
                    date_end=end_date,
                    metrics=[MetricConfig(metric=metric_type)],  # Use CORRECT format: metric=
                )

data_points = len(metrics.data) if metrics.data else 0

print(f"     📊 {data_points} data points")

     📊 1 data points
